# Changelogs

| Version  | Loss | EM Score | F1 Score | Changes | Comment |
|----------|----------|----------|--------------|---------|---------|
|  |  |  |    |   |  |

In [1]:
PATH='/home/ubuntu/kobigbird_final'
Bigbird_workspace = '/KoBigBird'

In [2]:
%cd $PATH

/home/ubuntu/kobigbird_final


In [19]:
!git clone https://github.com/katie0809/KoBigBird.git
%cd $PATH$Bigbird_workspace
!git pull

Cloning into 'KoBigBird'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 115 (delta 29), reused 101 (delta 20), pack-reused 0
Receiving objects: 100% (115/115), 382.21 KiB | 11.94 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/home/ubuntu/kobigbird_final/KoBigBird
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), 788 bytes | 788.00 KiB/s, done.
From https://github.com/katie0809/KoBigBird
   a5935cf..2f3348e  master     -> origin/master
Updating a5935cf..2f3348e
Fast-forward
 finetune/data/cls.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [20]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	KoBigBird/

nothing added to commit but untracked files present (use "git add" to track)


# Load & Prepare

In [ ]:
!pip install transformers==4.11.3
!pip install sentencepiece
!pip install -qr $PATH$Bigbird_workspace'/finetune/requirements.txt'
# !pip install torch==1.8.1
# !pip install git+https://github.com/vasudevgupta7/transformers.git@add_big_bird # TODO: replace with new pip version eventually
# !sudo apt-get install liblzma-dev

In [15]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import seaborn as sns
from sklearn.model_selection import train_test_split
import json
from tqdm import tqdm
import re
import datetime
import sys
import hashlib
import pickle
import yaml

import torch
from torch.utils.data import Dataset, DataLoader
import torchtext

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel,BertConfig
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering

import torch
import torch.utils.data as torch_data

# ignore warnings filter
from warnings import simplefilter
simplefilter('ignore')

global bert_inputs, docs, qnas

pd.set_option('display.max_colwidth', None)

2021-11-16 00:20:09.324746: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [16]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(keras.__version__)
print(sklearn.__version__)
print(transformers.__version__)
# print(pororo.__version__)
print(torch.__version__)

2.3.0
1.18.5
1.3.4
2.4.0
0.24.1
4.11.3
1.7.1


In [17]:
model_name = "monologg/kobigbird-bert-base"

data_dir = PATH+'/output'
train_file = data_dir + '/train.csv'
test_file = data_dir + '/test.csv'
samplesub_file = data_dir + '/sample_submission.csv'

build_dir = PATH+'/build'
# output_dir = build_dir / model_name
# trn_encoded_file = output_dir / 'trn.enc.joblib'
# val_predict_file = output_dir / f'{model_name}.val.txt'
submission_file = 'sub.csv'

id_col = 'id'
text_col = 'excerpt'
target_col = 'target'

max_len = 1024
ans_max = 200 # 답변 최대 글자수는 200
epochs = 1
n_fold = 5
n_est = 9
n_stop = 2
batch_size = 8
seed = 42

now = datetime.datetime.now()
year, month, day, hour, min = now.year, now.month, now.day, now.hour, now.minute
display(f'{month}월 {day}일 {hour}시 {min}분 실행')

version = f'{day}{hour}'

'11월 16일 0시 20분 실행'

# Train

In [ ]:
!python $PATH$Bigbird_workspace'/finetune/run.py' \
--task 'qa' \
--dataset 'korquad_2' \
--do_train \
--data_dir $PATH \
--train_file 'x_train1_.csv' \
--predict_file 'x_valid1_.csv' \
--model_name_or_path 'monologg/kobigbird-bert-base' \
--output_dir $PATH'/output' \
--data_dir $PATH \
--learning_rate 3e-5 \
--weight_decay 2e-5 \
--num_train_epochs 1 \
--train_batch_size 1 \
--max_seq_length 4096 \
--doc_stride 512 \
--max_answer_length 4096 \
--gradient_accumulation_steps 4

2021-11-16 00:24:14.782290: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/home/ubuntu/.local/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
11/16/2021 00:24:16 - ERROR - __main__ -   Failed to import XLA. No module named 'torch_xla'
Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.pooler.bias', 'bert.pooler.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expect

In [57]:
import pandas as pd

valid2 = pd.read_csv('x_valid1_.csv')
valid2

,title,text,question,answer,answer_start,answer_end
0,대학생들의 주관적 규범이 창업의도에 미치는 영향: 자기효능감의 매개효과와 안정지향성...,"을 증가시켰으며, 자기효능감은 창업의도에 영향을 주는 것으로 나타나 주관적 규범이 ...",경력지향성을 형성시키는 것은 무엇인가?,"자신의 재능, 욕구, 동기, 태도 및 가치관 등이 통합되어 형성된 직업 관련 이미지...",201,280
1,"제품, 공정, 서비스 통합 설계를 지원하는 제품자료모델","높아짐에 따라, 재료 사용(제품 설계), 생산(공정), 그리고 고객 서비스(사용 ...",기업이 관심을 가지는 제품 수명주기는 무엇으로 구성되어 있는가?,"제품 설계, 생산, 그리고 고객 서비스로 구성되어 있다",116,146
2,걸음마기 아동의 어머니를 대상으로 한 부모존경-자녀존중 부모교육 프로그램의 효과,stress;social and emotional behaviors; 본문 부모의 ...,어린 시기의 부모역할이 더욱 중요한 이유는?,생애 초기의 부모자녀 간 상호작용의 질은 인간발달의 기초를 형성하므로,190,228
3,파골세포 분화에서 토사자 물 추출물의 효과,be a promising drug for use against bone disor...,골의 유지는 어떻게 조절되는가?,파골세포가 오래된 골을 흡수하고 조골세포가 새로운 골을 꾸준하게 생산함으로써 조절된다,147,194
4,안면이식에 대한 최근 동향: 한국에서의 안면이식은 어떤 단계에 있는가?,"년 현재까지 총 21건의 안면이식이 시행되었다. 특히, 2009년에 6건이 시행된 ...",안면이식을 시행하기 위해 실질적으로 필요한 의학 분야는 어떻게 나눌 수 있는가?,"첫째, 미세수술을 통한 재건수술, 둘째, 면역억제제 치료와 관련된 이식 분야, 셋째...",98,176
...,...,...,...,...,...,...
33241,유방암 환자와 배우자의 갈등해결방식과 부부친밀도 및 가족기능,"밀감과 밀접한 관련성이 있는 것으로 간주된다(Lee & Ok, 2002). 부부간의...",부부간의 친밀감이란 어떤 개념인가?,"부부가 서로에게 느끼는 매우 가깝고도 공유되는 밀접함을 의미하며, 상호관계의 질을 ...",52,105
33242,500 PS급 선박 SCR 반응기에서 디퓨저 각도와 면적비에 따른 유동균일도 수치해석,"; 본문 디젤엔진은 주로 시간당, 마력당 비교적 낮은 소비율로 저급 연료를 사용할 ...",디젤엔진의 단점은?,"질소산화물(NOx), 미세먼지 등은 디젤엔진이 가솔린엔진보다 최대 100배 이상 더...",299,369
33243,정지 및 저속 하강 비행하는 헬리콥터 로터의 소음 해석 및 검증,로터 시스템에서는 복잡한 공력 특성이 나타나고 이로 인한 공력소음이 크게 발생하는 ...,회전익기의 소음 문제를 해결하기 위한 노력에는 어떤 것이 있는가?,블레이드 끝단의 형상을 변화시켜 블레이드-와류 간섭 소음을 저감시키는 등의 수동적인...,265,414
33244,80년대 표준설계 초등학교 단위 교실의 환경친화적 리노베이션에 관한 연구,0s. 핵심 어휘 표준설계;환경친화;자연채광;자연환기;리노베이션; Standard ...,우리나라 초등학교 시설은 대부분 언제 지어졌는가?,1985년 이전 양적 팽창 시기,124,141


# Backup

In [48]:
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained(
            'monologg/kobigbird-bert-base', 
        )
model = AutoModelForQuestionAnswering.from_pretrained(
            '/home/ubuntu/2021AiHub-ODQA/models/korquad_2/0-0-ckpt', 
        )

In [30]:
c = [
    '주차장에서 한 여성이 피투성이가 된 채 발견된다. 피해자는 니콜 매닝. 놀랍게는 니콜은 임산부였고 누군가에게 폭행당한 후 제왕절개술로 아이를 뺏긴 채 쓰러져 있었다. 니콜에게 원한이 있는 사람을 중심으로 수사하던 성범죄 전담반은 니콜이 간호사로 일한 메타딘 클리닉의 환자였던 카일 노바첵과 니콜에게 부정 행위를 들켜 실직된 전직 간호사 에린 두 사람을 의심한다. 하지만 카일은 완벽한 알리바이를 제공하고 뭔가를 숨기는 듯한 에린 역시 더 이상 수사에 도움이 되지는 않는다. ',
    "The Ever Given is 400m-long (1,312ft) and weighs 200,000 tonnes, with a maximum capacity of 20,000 containers. It is currently carrying 18,300 containers."
]
q = [
    '피해자의 이름은 무엇인가?',
    "How heavy is Ever Given?"
]

In [58]:

encodings = tokenizer(valid2['question'].iloc[10], 
                    valid2['text'].iloc[10], 
                    return_tensors="pt")

# input 을 cuda로 저장
print("type ",type(encodings))
print(encodings)

# input_ids = encodings["input_ids"].to(self.device).cpu()
# token_type_ids = encodings["token_type_ids"].to(self.device).cpu()
# attention_mask = encodings["attention_mask"].to(self.device).cpu()
input_ids = encodings["input_ids"]
token_type_ids = encodings["token_type_ids"]
attention_mask = encodings["attention_mask"]

type  <class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': tensor([[    2, 13070,  4771,  4828, 31318,  9356,  4770,  4898,  8657,  4741,
          6907,  7721,  7605,  4584,  7136,  4604,  3246,  4696,  2935,  4662,
          4649,   535,     3, 10730,  5045,  4293,  9954,  5189, 12853, 13555,
          5189,  4521,  3573,  4741,  9356,  4770,  4898,  4839, 15032,  4584,
          4322,  5246,  2935, 12853, 15032,  4598,  4584,  7131,  4681,  4990,
         19137,  4618,  9888,  5189,  4521,  3573,  4839,  3913, 13070,  4771,
          4828,  2776,  4584,  7954,  5547,  7581,  3738,  4742,   518,  7010,
          4837, 13070,  4771,  4828, 31318,  4561,  9356,  4770,  4898,  8657,
          4741, 10730,  5189,  4741,  9356,  4770,  4898,  4561,  8011,   516,
          9356,  4770,  4898,  4561, 13181,   516, 15032,  4561,  8570, 24158,
          4809,  8011,  4676,  2556,  4839,  9356,  4770,  4898,  4676, 15032,
          4542,  4602,   516,  9231,  4561,  9356,

In [59]:
outputs = model(input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)

In [60]:
start_logits, end_logits = outputs[0], outputs[1]
token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
pred = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encodings["input_ids"][0][token_start_index: token_end_index + 1]))

In [61]:
pred

'[CLS]'